In [60]:

import pandas as pd
import numpy as np
import re

In [61]:
import requests
from io import StringIO

url = "https://api.data.abs.gov.au/data/ABS%2CABS_NOM_VISA_CY%2C1.0.0/1%2B2%2B3%2B4%2B5%2B6%2B7%2B8%2B9%2B10%2B11%2B12%2B13%2B14%2B15%2B16%2B17.1.AUS.A?detail=full"
headers = {"accept": "text/csv"}
req = requests.get(url, headers=headers)
data = StringIO(req.text)
absdf= pd.read_csv(data)
absdf = absdf.loc[absdf['MEASURE'] == 16]
absdf = absdf[['TIME_PERIOD','OBS_VALUE']]
absdf.rename(index=str, columns={"TIME_PERIOD": "year"}, inplace=True)
absdf.rename(index=str, columns={"OBS_VALUE": "migration_no"}, inplace=True)
absdf.to_csv("./datasets/cleaned-migration-years.csv", index = False)

In [62]:
# https://www.data.qld.gov.au/dataset/register-of-immigrants-1864-to-1878
url = "https://www.data.qld.gov.au/dataset/32e5cda7-dee5-4cc2-a5f5-88b9a04da882/resource/0c976207-a184-4605-b171-a65a76a2ad2e/download/register-of-immigrants-1864-1878.csv"
# url = "./datasets/register-of-immigrants-1864-1878.csv"

qdf= pd.read_csv(url )

qdf['Description'].replace(to_replace="(.*from\s)(.*)", value=r"\2", regex=True, inplace=True)
qdf.rename(index=str, columns={"Description": "country"}, inplace=True)
qdf.dropna(subset=['country' ], inplace=True) 

qdf.rename(index=str, columns={"Date of arrival": "year"}, inplace=True)

qdf['year'].replace(to_replace=".*([0-9]{4}).*", value=r"\1", regex=True, inplace=True)
qdf['year'].nunique()

qdf = qdf.sort_values(by=['year'], ascending = False)
s = qdf.groupby(['country','year']).size()
qdf = s.to_frame(name = 'migration_no').reset_index()

In [63]:


# url = "https://data.gov.au/data/dataset/2fe5e2a9-8a3d-4dcf-baec-c5147d953150/resource/b59a15df-86ea-4c4c-95be-4dd9fc9f8ac4/download/2019-20-historical-migration-statistics-locked.xlsx"
url = "./datasets/2019-20-historical-migration-statistics-locked.xlsx"

# Table 1.1: Permanent and long-term arrivals, October 1945 to June 1959
df1= pd.read_excel(url, sheet_name="1.1", header=7 )
# Table 1.2: Settler arrivals, January 1959 to June 1975  
df2= pd.read_excel(url, sheet_name="1.2", header=7 )
#Table 1.3: Settler arrivals, 1975–76 to 1994–95  
df3= pd.read_excel(url, sheet_name="1.3", header=7 )
#Table 1.4: Settler arrivals, 1995–96
df4= pd.read_excel(url, sheet_name="1.4", header=7 )
# Table 2.1: Permanent additions, 1996–97 to 2007–08
df5= pd.read_excel(url, sheet_name="2.1", header=7 )
# Table 2.2: Permanent additions, 2008–09 to 2016–17
df6= pd.read_excel(url, sheet_name="2.1", header=7 )

#Table 3.2: The permanent migration program outcome by stream and citizenship, 1996–97 to 2016–17
df7= pd.read_excel(url, sheet_name="3.2", header=7 )


frames = [df1, df2, df3, df4, df5, df6, df7]

stores_df = pd.concat(frames)


In [64]:


stores_df = stores_df.iloc[: , :-1]
stores_df = stores_df.iloc[: , 1:]
stores_df = stores_df.melt(id_vars=["Year"], 
        var_name="country", 
        value_name="migration_no")
stores_df.rename(index=str, columns={"Year": "year"}, inplace=True)

stores_df.dtypes

year            object
country         object
migration_no    object
dtype: object

In [65]:
stores_df = pd.concat([qdf,stores_df], axis=0)


In [66]:
stores_df.dropna(subset=['migration_no', 'country' ], inplace=True) 

stores_df['year'].replace(to_replace=".*([0-9]{4}).*", value=r"\1", regex=True, inplace=True)

stores_df['migration_no'].replace(to_replace="\D*(\d*)", value=r"\1", regex=True, inplace=True)
stores_df[['year','migration_no']] = stores_df[['year','migration_no']].astype(int)
stores_df= stores_df[stores_df['migration_no'] != 0]

stores_df = stores_df[~stores_df.country.str.contains('|'.join(['Australia', 'Other', 'total', 'Total']))]

stores_df = stores_df.sort_values(by=['year', 'migration_no'], ascending = False)
stores_df = stores_df.drop_duplicates(subset=['year', 'country'])


In [67]:
remove = ['.1', '.2', '.3', '.4', '']
rexpress = '[' + re.escape (''. join (remove)) + ']'
stores_df['country'] = stores_df['country'].str.replace(rexpress, '', regex=True)

remove_words = [', Dem Peoples Rep Of', ', People\'s Republic of', 'and Ireland', '\x28excludes SARs and Taiwan\x29', '\x28SAR of China\x29']
rexpress = r'\b(?:{})\b'.format( '|'.join(remove_words))
stores_df['country'] = stores_df['country'].str.replace(rexpress, '', regex=True)

stores_df['country'].replace(["U.S.S.R."], "Russia",  inplace=True)
# stores_df[stores_df['country'].str.contains("SAR of China")]

stores_df = stores_df.sort_values(by=['year', 'migration_no'], ascending = True)
stores_df = stores_df.drop_duplicates(subset=['year', 'country'])
stores_df


,country,year,migration_no
15,http://wwwarchivessearchqldgovau/Search/ItemDe...,43,1
0,Ireland,1864,121
1,Ireland,1865,82
2,Ireland,1866,49
3,Ireland,1867,25
...,...,...,...
47693,Colombia,2016,438
26867,Russian Federation,2016,461
46981,Brazil,2016,574
2659,France,2016,672


In [68]:


import pycountry
list_countries = stores_df['country'].unique().tolist()
d_country_code = {} 
for country in list_countries:
    try:
        country_data = pycountry.countries.search_fuzzy(country)
        country_code = country_data[0].alpha_3
        d_country_code.update({country: country_code})
    except:
        print('\n could not add ISO 3 code for:', country)
        d_country_code.update({country: np.nan})
print(d_country_code)


 could not add ISO 3 code for: http://wwwarchivessearchqldgovau/Search/ItemDetailsaspx?ItemId=769

 could not add ISO 3 code for: USSR

 could not add ISO 3 code for: Yugoslavia

 could not add ISO 3 code for: Czechoslovakia

 could not add ISO 3 code for: Unnamed: 68

 could not add ISO 3 code for: At Sea & Not Stated

 could not add ISO 3 code for: Unnamed: 77

 could not add ISO 3 code for: Micronesia nfd

 could not add ISO 3 code for: St Helena

 could not add ISO 3 code for: Swaziland

 could not add ISO 3 code for: Netherlands Antilles

 could not add ISO 3 code for: South America nfd

 could not add ISO 3 code for: Southeast Asia nfd

 could not add ISO 3 code for: Samoa, Western

 could not add ISO 3 code for: Southern and East Africa nfd

 could not add ISO 3 code for: The Caribbean nfd

 could not add ISO 3 code for: Macau

 could not add ISO 3 code for: Taiwan (Province of China)

 could not add ISO 3 code for: Laos

 could not add ISO 3 code for: China (excluding Taiwan P

In [69]:
for k, v in d_country_code.items():
    stores_df.loc[(stores_df.country == k), 'iso_alpha'] = v

stores_df.dropna(subset=['iso_alpha' ], inplace=True) 

stores_df

,country,year,migration_no,iso_alpha
0,Ireland,1864,121,IRL
1,Ireland,1865,82,IRL
2,Ireland,1866,49,IRL
3,Ireland,1867,25,IRL
4,Ireland,1868,5,IRL
...,...,...,...,...
47693,Colombia,2016,438,COL
26867,Russian Federation,2016,461,RUS
46981,Brazil,2016,574,BRA
2659,France,2016,672,FRA


In [70]:
stores_df.to_csv("./datasets/cleaned-migration.csv", index = False)